[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Operacional-III-A/blob/main/11_Fun_Geradora.ipynb)

## **Pesquisa Operacional III-A**

**Professor:**
- Diogo Ferreira de Lima Silva (TEP-UFF)


# Funções Geradoras

Uma função geradora é uma função especial que tem como característica retornar, cada vez que é chamada, valores em sequência. 

- Uma função em Python passa a ser geradora quando o comando **yield** é utilizado.

Na primeira vez em que a função é chamada, o **yield** funciona da mesma forma que o **return**.

**Porém, a partir da segunda vez, uma característica especial é ativada: a execução iniciará na linha após o **yield** !!!**

Vejamos inicialmente o funcionamento de uma função convencional:

In [ ]:
def conta_convencional ():
    conta = 0
    while True:
        return conta
        conta +=1

In [ ]:
minha_conta = conta_convencional()

print(minha_conta)
print(minha_conta)
print(minha_conta)
print(minha_conta)

Agora, trabalhando com uma função geradora, perceba que o valor de conta continuará sendo atualizado!

In [ ]:
def conta_geradora ():
    conta = 0
    while True:
        yield conta
        conta +=1


O método **next** executa a função até o próximo **yield** retornar algo.

In [ ]:
minha_conta = conta_geradora()

print(minha_conta)
print(next(minha_conta))
print(next(minha_conta))
print(next(minha_conta))
print(next(minha_conta))

# Exemplo de uso

Vamos criar um processo simples apenas para exemplificar o uso da função geradora.

A aula do Jupyter Notebook **2_Intro_Simpy.ipynb** trará mais detalhes sobre o uso da biblioteca.

## Criando um Processo Simples

Vamos criar um processo com quatro atividades: A -> B -> C -> D.

- Apenas 40% dos trabalhos passam por B. Os outro 60% saem de A direto para C.
- Os tempos de processamento são, respectivamente, 10 25, 8, 5.
- As atividades A, B e C usam o recurso R1.
- A atividade D usa o recurso R2.




In [ ]:
# Caso esteja no Google Colab
!pip install simpy

Inicialmente, vamos importar algumas bibliotecas que serão utilizadas.

In [ ]:
import simpy # Biblioteca para modelagem e simulação de processos
import random # Biblioteca para gerar números aleatórios
import numpy as np # Biblioteca para trabalhar com operações em vetores e matrizes

In [ ]:
random.seed(10) # Gerando uma semente para a geração de números aleatórios

### Etapa 1

Inicialmente, vamos modelar cada uma das atividades do nosso processo:

In [ ]:
def atividade_A (environment, trabalho_id):
    # Para a atividade A ser executada, precisamos "esperar" por um recurso R1
    with R1.request() as req:
        yield req # Tempo passa até R1 estar disponível
        yield environment.timeout(10) # Tempo de processamento da atividade A
        
def atividade_B (environment, trabalho_id):
    # Atividade B requer R1
    with R1.request() as req:
        yield req # Tempo passa até R1 estar disponível
        yield environment.timeout(25)  # Tempo de processamento da atividade B
        
def atividade_C (environment, trabalho_id):
    # Requerimento de R1
    with R1.request() as req:
        yield req # Tempo passa até R1 estar disponível
        yield environment.timeout(8)  # Tempo de processamento da atividade C
        
def atividade_D (environment, trabalho_id):
    # Requerimento de R2
    with R2.request() as req:
        yield req # Tempo passa até R2 estar disponível
        yield environment.timeout(5) # Tempo de processamento da atividade D
         

### Etapa 2


Agora, vamos criar uma função para o nosso processo. 

Em outras palavras, o que acontece com um trabalho inicializado no processo.

Perceba que dessa vez passamos a palavra "ambiente" como parâmetro ao invés de "environment". 

Isso é apenas para ilustrar que na criação das funções, essa é uma escolha do usuário. O importante é que a palavra usada como input após o nome da função seja utilizada ao longo da mesma. Ou seja, dentro da função devemos continuar com o padrão "ambiente".

In [ ]:
# Modelando o processo

def primeiro_processo (ambiente, trabalho_id, R1, R2):
        #print(f'tempo: {ambiente.now} -- {trabalho_id} entrou no processo')
        entrou_no_processo = ambiente.now #entrou_no_processo recebe o tempo de entrada do processo
        
        # Atividade A
        yield (ambiente.process(atividade_A(ambiente, trabalho_id)))
   
        # Atividade B só acontece em 40% dos trabalhos
        regra_B = random.uniform(0,1) 
        if regra_B < 0.4:
                yield (ambiente.process(atividade_B(ambiente, trabalho_id)))
    
        # Atividade C
        yield(ambiente.process(atividade_C(ambiente, trabalho_id)))

        # Atividade D
        yield(ambiente.process(atividade_D(ambiente, trabalho_id)))

        saiu_do_processo = ambiente.now #entrou_no_processo recebe o tempo de saída do processo
        
        TC = saiu_do_processo - entrou_no_processo # TC é o tempo de ciclo

        tempo_de_ciclo.append(TC) # Inclui o TC observado na lista de tempos de ciclo 
        

### Etapa 3

Agora vamos modelar a chegada dos trabalhos no processo. 

In [ ]:
def chegadas (ambiente):
    id = 1 #guarda o id do cliente 
    # Enquanto houver simulação:
    while True:
        # Passa um tempo até a próxima chegada
        yield ambiente.timeout(50)
        
        # Um cliente chega no processo
        ambiente.process(primeiro_processo (ambiente, 'Trabalho %d' % id, R1, R2))
        
        # O próximo cliente terá id = id + 1
        id += 1

In [ ]:
tempo_de_ciclo = [] # Vamos criar uma lista vazia que receberá os tempos de ciclo observados
env = simpy.Environment()
R1 = simpy.Resource(env, capacity=1)
R2 = simpy.Resource(env, capacity=1)
env.process(chegadas(env))
env.run(until = 100000)


In [ ]:
np.mean(tempo_de_ciclo)

In [ ]:
tempo_de_ciclo

In [ ]:
TC = []
for i in range(100):
    tempo_de_ciclo = []
    env = simpy.Environment()
    R1 = simpy.Resource(env, capacity=1)
    R2 = simpy.Resource(env, capacity=1)
    env.process(chegadas(env))
    env.run(until = 100000)
    TC.append(np.mean(tempo_de_ciclo))

In [ ]:
np.mean(TC)